In [5]:
import yfinance as yf
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import pandas_market_calendars as mcal
from datetime import datetime

# --------- Configuration ---------
stock_symbol = "RELIANCE.NS"
start_date = "2024-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")

# --------- Fetch Data ---------
df = yf.download(stock_symbol, start=start_date, end=end_date)

# Check if data was returned
if df.empty:
    print("No data found for the given symbol and date range.")
    exit()

df.reset_index(inplace=True)

# --------- Clean & Rename Columns ---------
df = df.rename(columns={
    "Date": "Date",
    "Open": "Open",
    "High": "High",
    "Low": "Low",
    "Close": "Close",
    "Volume": "Volume"
})
df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]

# --------- Add Month Column ---------
df["Month"] = df["Date"].dt.strftime("%Y-%m")

# --------- Get NSE Calendar ---------
nse = mcal.get_calendar("NSE")
schedule = nse.schedule(start_date=start_date, end_date=end_date)

# --------- Expiry Logic: Last Thursday that's a trading day ---------
def get_expiry_with_holidays(year, month, schedule):
    last_day = pd.Timestamp(year=year, month=month, day=1) + MonthEnd(0)
    for i in range(7):
        day = last_day - pd.Timedelta(days=i)
        if day.weekday() == 3 and day in schedule.index:  # Thursday and trading day
            return day
    return pd.NaT

# Generate expiry mapping
unique_months = pd.date_range(start=start_date, end=end_date, freq='MS')
expiry_dates = {
    dt.strftime("%Y-%m"): get_expiry_with_holidays(dt.year, dt.month, schedule)
    for dt in unique_months
}

# --------- Add Expiry Column ---------
df["Expiry"] = df["Month"].map(expiry_dates)

# --------- Save to CSV ---------
output_file = f"{stock_symbol}_data_with_expiry.csv"
df.to_csv(output_file, index=False)
print(f"CSV file saved as: {output_file}")


[*********************100%***********************]  1 of 1 completed

CSV file saved as: RELIANCE.NS_data_with_expiry.csv


In [6]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import os

# Path to your existing CSV
input_file = "RELIANCE.NS_data_with_expiry.csv"  # <-- Replace this with your actual file name
output_file = "RELIANCE_absMove.csv"

# Read CSV and convert 'Date' column to datetime
df = pd.read_csv(input_file, parse_dates=["Date"])

# Standardize column names if needed
df.rename(columns=lambda x: x.strip().capitalize(), inplace=True)

# Step 1: Calculate expiry date (last Thursday of each month)
def get_last_thursday(date):
    last_day = date + MonthEnd(0)
    while last_day.weekday() != 3:  # Thursday is 3
        last_day -= pd.Timedelta(days=1)
    return last_day

df["Expiry"] = df["Date"].apply(get_last_thursday)

# Step 2: Get expiry close value
expiry_close_map = df.groupby("Expiry")["Close"].last().reset_index()
expiry_close_map.rename(columns={"Close": "Expiry_Close"}, inplace=True)

# Step 3: Merge expiry close back into original dataframe
df = pd.merge(df, expiry_close_map, on="Expiry", how="left")

# Step 4: Calculate absolute move
df["Absolute_Move"] = (df["Close"] - df["Expiry_Close"]).abs()

# Step 5: Save to new CSV
df.to_csv(output_file, index=False)
print(f"Updated data saved to {output_file}")


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import os

# ---------- Load your stock CSV ----------
file_path ="RELIANCE.NS_data_with_expiry.csv"  # Change if needed
df = pd.read_csv(file_path, parse_dates=["Date"])

# Ensure Date is sorted
df.sort_values("Date", inplace=True)
df.reset_index(drop=True, inplace=True)

# ---------- Add Month ----------
df["Month"] = df["Date"].dt.to_period("M")

# ---------- Function to get expiry date (last Thursday of the month) ----------
def get_expiry(month_period):
    # Convert period to timestamp (end of month)
    last_day = month_period.end_time
    while last_day.weekday() != 3:  # Thursday is weekday 3
        last_day -= pd.Timedelta(days=1)
    return last_day

# Get unique expiry dates for all months
expiry_df = pd.DataFrame()
expiry_df["Month"] = df["Month"].unique()

# Convert "Month" back to timestamp to calculate expiry
expiry_df["Expiry"] = expiry_df["Month"].apply(lambda x: get_expiry(x))

# ---------- Merge expiry date to main DataFrame ----------
df = df.merge(expiry_df, on="Month", how="left")

# ---------- Calculate absolute move from date to expiry ----------
# Create a dict of Close prices on expiry for fast lookup
expiry_close_dict = df[df["Date"] == df["Expiry"]].set_index("Month")["Close"].to_dict()

# Map expiry close to each row
df["Expiry_Close"] = df["Month"].map(expiry_close_dict)

# Compute absolute move (expiry close - today's close)
df["Abs_Move"] = (df["Expiry_Close"] - df["Close"]).abs()

# ---------- Save to CSV ----------
output_file = "stock_data_with_expiry_and_move.csv"
df.to_csv(output_file, index=False)
print(f"✅ File saved as {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: 'stock_data.csv'